# P1: Autoencoder 

We discuss in class that autoencoder can automatically discover the binary code for decimal numbers. The case we consider in class is the binary code for 8 decimal numbers from 0 to 7. Recall that in this autoencoder we use a position encoding scheme (check your class notes to see what that is). 
We now construct a few autoencoders for finding binary code for 16 decimal numbers from 0 to 15. We are going to use the same position encoding scheme here.  
For these autoencoders, the input layers and the output layers all have 16 nodes. We will use the sigmod function (i.e., f(x) = 1/(1+e^(-x))) for perceptron activation and consider the following architectures (for problems 1-4):

1.	A 3-layer NN (with layers for input, hidden and output). The hidden layer has 5 perceptrons. 
2.	A 3-layer NN. The hidden layer has 4 perceptrons.
3.	A 3-layer NN. The hidden layer has 3 perceptrons. 
4.	A 5-layer NN. The 1st, 2nd, and 3rd hidden layers have 8, 4, and 8 perceptrons, respectively. 
5.	Repeat 4) above, but this time we use the Rectified Linear Unit (ReLU), i.e., f(u) = max(0, u), for all perceptron activation. Compare the average running time of 5 runs of the two methods (correspondingly, each with the same initial random weights). (5 pts)

What to report: Run your program 5 times with different initial weights and compare the stable states of all hidden layers from all these 5 runs on each autoencoder architecture. Report and compare the stable states of all hidden layers after you train the autoencoders using different initial weights. Discuss the correspondence of these states and the input values.  For problem 5) above, compare and discuss the stable states using the two different activation functions. If you cannot get stable states for one or both of these activation functions, explain the reason(s) why you cannot have stable states.



# 0. Generating data and implementing helper functions

In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import math

In [2]:
ones = np.ones(16)
x = np.diag(ones)

In [3]:
print x

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. 

In [4]:
y = x

** Discussion **

For this homework, I used sklearn Multi-layer Perceptron classifier. For autoencoder, the input and output should be the same. Each element in x represents one number. For example, we could treat the first line printed above as 1, second line as 2, and so on. The sklearn model optimizes the log-loss function using LBFGS or stochastic gradient descent.

In a typical run, I have the following codes.
```python
clf = MLPClassifier(activation = activation_function, learning_rate_init= 0.01, alpha=1e-5, hidden_layer_sizes= layers ,max_iter=200000, random_state=i)
clf.fit(x,y)
```

parameters : 

* activation: activation function for the hidden layer. ‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)); ‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
* learning_rate_init: the initial learning rate used. It controls the step-size in updating the weights. 
* alpha: L2 penalty (regularization term) parameter.
* hidden_layer_sizes:  tuple, length = n_layers - 2, default (100,)mThe ith element represents the number of neurons in the ith hidden layer.
* max_iter: maximum number of iterations.
* random_state: if int, random_state is the seed used by the random number generator. In this problem , I used 0,1,2,3,4 to seed the 5 intial random weights.


Results:

* loss = clf.loss_  : loss computed with the loss function.
* score = clf.score(x,y) : returns the mean accuracy on the given test data and labels. In this case, it represents the training accuracy.
* weights = clf.coefs_ [0] : clf.coefs_ returns the ith element in the list represents the weight matrix corresponding to layer i. In 3-layer NN, clf.coefs_ [0] represents the weight from input layer to hidden layer.
* bias = clf.intercepts_[0] : clf.intercepts_ returns the ith element in the list represents the bias vector corresponding to layer i + 1.


Helper function:

```python
def compare(layers, activation_function, printHidden, learning_rate):
```
Compare function will compare the results of each NN with 5 different intial weights. It will gives loss and training accuracy of the model. Inside the function, I also called get_hiddenLayer_output() and convert_output_to01() functions to print out the hidden layer outputs(both original and converted) for all 16 numbers. I also checked if the binary code in hidden layer could represent all the numbers or not.


In [5]:
# sigmoid function
def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))

In [9]:
# The function helps to compare the results of NN with 5 different intial weights.
def compare(layers, activation_function, printHidden, learning_rate):
    #parameter:
    # layers: tuple, a list of perceptron numbers in hidden layers
    # activation_function: string, the activation function passed into the MLPClassifier
    # printHidden, boolean, if true, print out the hidden layer outputs (original and converted)
    # learning rate : the learning rate passed into the MLPClassifier
    
    # in each iteration, generate one set of seeded intial wieghts
    for i in range(5):
        clf = MLPClassifier(activation = activation_function, learning_rate_init= learning_rate, alpha=1e-5, hidden_layer_sizes= layers ,max_iter=200000, random_state=i)
        clf.fit(x, y)
        loss = clf.loss_
        score = clf.score(x,y)
        weights = clf.coefs_ [0]
        bias = clf.intercepts_[0]
        print "random intialization ", i + 1, ": ", "score is ", score, " ; loss is " , loss
        
        #print out the hidden layer outputs
        if printHidden:
            #original hidden layer outputs
            print "hidden layer output:"
            hidden_output = get_hiddenLayer_output(x, weights, bias)
            #convert hidden layer outputs to 0/1 for discussion simplicity
            print "convert hidden layer output to 0/1:"
            coverted_hidden_output = convert_output_to01(hidden_output)
            #print out the duplicate binary code in hidden layer
            duplicates = check_binary(coverted_hidden_output)
            if len(duplicates) == 0:
                print "16 numbers is successfully represented using different binary code in the hidden layer."
            else:
                print "Failed to represent 16 numbers using different binary code in the hidden layer."
                print "Duplicates:", duplicates
                
        #print out the final layer output 
        predicts = clf.predict(x)
        print "final output"
        print predicts
        print "Accuracy: ", clf.score(x, y)
        print ""

In [10]:
# The function computes the hidden layer outputs
def get_hiddenLayer_output(x, weights, bias):
    output = []
    # loop through all the data points
    for i in range(len(x)) :
        total = []
        # for each data point, compute its corresponding hidden layer outputs with trained weights and bias
        for weight, b in zip(weights.T,bias.T): 
            total.append(sigmoid(np.dot(x[i], weight) + b))
        # print out the original data and hidden layer outputs
        print i , ": ", total
        output.append(total)
    return output

In [11]:
# The function conver the hidden layer outputs into 0/1 for discussion simplicity
def convert_output_to01(output):
    new_output = []
    for i in range(len(output)):
        new_o = []
        for num in output[i]:
            if num > 0.5:
                new_o.append(1)
            else:
                new_o.append(0)
        print i , ": ", new_o
        new_output.append(new_o)
    return new_output

In [12]:
# The function checks to see if the hidden layer achieves successful binary representatoin
# return the duplicates 
def check_binary(output):
    
    #convert the list of 0/1 to a string to represent the number
    new_output = [str(o) for o in output]  
    from collections import Counter
    duplicates = [k for k,v in Counter(new_output).items() if v>1]
    return duplicates

# 1. 3-layer NN with 5 perceptrons in the hidden layer

In [13]:
compare((5,) , 'logistic', True, 0.01)

random intialization  1 :  score is  1.0  ; loss is  0.137348771489
hidden layer output:
0 :  [0.9972535907544947, 0.000929066632688839, 0.9977038356245037, 0.15958338126167376, 0.9956898015004597]
1 :  [0.9987519252402189, 0.9990010391742221, 0.9977091944714267, 0.8201641728789094, 0.0005687283904284266]
2 :  [0.0011717553543020343, 0.997699061924621, 0.998616709870235, 0.6259694732717773, 0.8842482966261132]
3 :  [0.9983763628461368, 0.0007353220129544848, 0.9972319092746398, 0.9962404732174293, 0.013468187891325636]
4 :  [0.9956865278932195, 0.4191724794637822, 0.06178262698521692, 0.9983028393553528, 0.01242805136878371]
5 :  [0.036170303981862106, 0.2760042584441363, 0.9989161459643173, 0.5868285133071445, 0.00565626637786696]
6 :  [0.8468067376374383, 0.9979168643140712, 0.9218298508861726, 0.001084355913936453, 0.9994341517949658]
7 :  [0.9933327294321476, 0.0007054880480198448, 0.15648473247266953, 0.9991401973896682, 0.9972383527212512]
8 :  [0.9969383345652046, 0.553113268373

** Discussion **

With learning rate set as 0.01, all 5 runs for NN(5 perceptrons in hidden layer) converges and gives very good training accuracy score (1.0) and very low loss( < 0.16). All five runs generated the final output exactly same as the input with accuracy as 1.0. The final output is stable.

The original hidden layer outputs are a set of real numbers in the range of 0 to 1 (see above tables). For each number, we got 5 corresponding stable states of hidden layer nodes, as shown in above section. For example, in first run, we got [0.9972535907544947, 0.000929066632688839, 0.9977038356245037, 0.15958338126167376, 0.9956898015004597] represents 0. These real numbers are close to either 1 or 0 with few exceptions. 


For comparing simplicity, I set a threshold of 0.5 to convert these real numbers into 0 or 1, i.e., any number bigger than 0.5 set as 1 and 0 otherwise. This would gives us the binary represenation of the input number in the hidden layer.

Now we could clearly see that inside each run except run 4, we achieves successfully binary representation of each number in the hidden layer, i.e., we could use different binary representation for different number. This makes sense since we only need 4 perceptrons to binarily represent 16 numbers. The only exception is run 4, where we use [1, 1, 0, 1, 1] represented both 8 and 12. The algorithm still converges and gives 100% accuracy possibly due the fact that in practice we used real number rather than binary number in hidden layer.

Between different runs, since we start with different intial weights, the final binary representation for a specific number may change. For example, we have [1, 0, 1, 0, 1] represents 0 in the first run and [1, 1, 1, 0, 0] represents 0 in the second run. Although both runs generated different binary representation for 0, they both successfullly achieved binary representation for 16 numbers in their own way. That is to say the sets of binary codes are the same among the runs. In this sense, I think the states of the hidden layer are stable.

# 2. 3-layer NN with 4 perceptrons in the hidden layer

In [105]:
compare((4,) , 'logistic', True, 0.01)

random intialization  1 :  score is  0.875  ; loss is  0.314742439679
hidden layer output:
0 :  [0.9985758138142072, 0.011365529725903415, 0.03986725213658222, 0.7020097843469328]
1 :  [0.08455481732657076, 0.85011406460693, 0.0013804303546094713, 0.46188754098595547]
2 :  [0.999490806778133, 0.0017059006960237, 0.9990359542107083, 0.9989584618046012]
3 :  [0.2701122656350515, 0.00029050775210826457, 0.9976241215451325, 0.5979422579406634]
4 :  [0.9958531479712722, 0.9978342076711461, 0.0006451087353640342, 0.9934537782458278]
5 :  [0.9770562097294104, 0.9982123599025229, 0.0010444136789722412, 0.04127654937409666]
6 :  [0.11307209592513297, 0.12160701761247149, 0.07048160352213113, 0.8175773974917608]
7 :  [0.00045897456416944316, 0.43540810713780326, 0.9978807033049315, 0.9981189937990026]
8 :  [0.9987070022985127, 0.9976430373365605, 0.00027203442810131687, 0.9985987607108713]
9 :  [0.0010489176489690145, 0.9981452605844843, 0.9994193548130402, 0.3318186297255382]
10 :  [0.999077828

With learning rate set as 0.01, all 5 runs for NN(4 perceptrons in hidden layer) converges and gives very good training accuracy and very low loss, though the result is not as good as the NN with 5 perceptrons in hidden layer. The final output is almost same as the input, with accuracy being 1.0(1 run) or 0.875(4 runs).The stable states of final output is different possibly due to the fact that with different random intialization the algorithm converged to different local minimum.

The original hidden layer outputs are a set of real numbers in the range of 0 to 1. For each number, I calculated 4 corresponding states of hidden layer nodes after the algorithm converges, as shown in above section. For example, in run 1, I got [0.9985758138142072, 0.011365529725903415, 0.03986725213658222, 0.7020097843469328] to represent 0 in the hidden layer. We could see these real numbers are close to 1 or 0 with some exceptions.

For comparing simplicity, we set a threshold of 0.5 to convert these real numbers into 0 or 1, i.e., any number bigger than 0.5 set as 1 and 0 otherwise. This would gives us the binary represenation of the input number in the hidden layer.

Now we could clearly see that inside each run, we achieves some binary representation of each number in the hidden layer. However, if we just used binary representation, the hidden layer fails to represent all 16 numbers, because some binary code is used to represent mulitple numbers. For example, in run 1, [0, 1, 0, 0] represented both 2 and 12. This happens to all 5 runs. Interestingly, we should be able to use 4 perceptrons to represent 16 numbers in theory.  The algorithm still converges possibly due the fact that in practice we used real number rather than binary number in hidden layer. 

Between different runs, since we start with different intial weights, the final binary representation in hidden layer for a specific number may change. For example, we have [1, 0, 0, 1] represents 0 in the first run and [1, 0, 0, 0] represents 0 in the second run. And it is clearly shown that lot of difference exists between runs regarding the binary representation in hidden layer.

# 3. 3-layer NN with 3 perceptrons in the hidden layer

In [106]:
compare((3,) , 'logistic', True, 0.01)

random intialization  1 :  score is  1.0  ; loss is  0.200107669423
hidden layer output:
0 :  [0.9997714609328417, 0.34513298724656155, 0.9997484394145042]
1 :  [0.9997489483045012, 0.17966474328738452, 0.049653542714081796]
2 :  [0.16349512647924785, 0.854989626884116, 0.82166899583689]
3 :  [8.636079573402618e-05, 0.6282484326952336, 0.34768061096276454]
4 :  [0.1597722433715804, 0.29486194281004835, 0.00016768273997130846]
5 :  [0.30625254640311766, 0.9998495213206119, 0.3727902594280726]
6 :  [0.7036247584680626, 0.9995409997189478, 0.9998505338021197]
7 :  [0.5020640702559874, 0.7780276731349842, 8.991892803906307e-05]
8 :  [0.44931625454394847, 0.00033051711173067626, 0.8567304013217348]
9 :  [0.9998507648620385, 0.9998588614181981, 0.4115150971486991]
10 :  [0.00020911638900628172, 0.23420055079781932, 0.5739178880185117]
11 :  [0.5592694704896163, 0.027801391396730428, 0.0002211371002827763]
12 :  [0.21440895234398963, 5.609858721299348e-05, 0.2877627045027906]
13 :  [0.1818966

With learning rate set as 0.01, all 5 runs for NN(3 perceptrons in hidden layer) converges and gives very good training accuracy and very low loss. The final output is the same as the input for all five runs with 1.0 accuracy. The final outputs are very stable. 

The original hidden layer outputs are a set of real numbers in the range of 0 to 1. For each number, I calculated 3 corresponding states of hidden layer nodes after the algorithm converges, as shown in above section. For example, in run 1, I got [0.9997714609328417, 0.34513298724656155, 0.9997484394145042] to represent 0 in the hidden layer. We could see these real numbers are close to 1 or 0 with quite a few exceptions.

For comparing simplicity, we set a threshold of 0.5 to convert these real numbers into 0 or 1, i.e., any number bigger than 0.5 set as 1 and 0 otherwise. This would gives us the binary represenation of the input number in the hidden layer.

Now we could clearly see that inside each run, we achieves some binary representation of each number in the hidden layer. However, the hidden layer fails to represent all 16 numbers, because some binary code is used to represent mulitple numbers. The case is worse than the one with 4 perceptrons or 5 perceptrons, which make sense since we could maximally represent 8 number with 3 perceptrons. For example, in run 1, [1, 0, 0] represented both 1 and 11. This happens to all 5 runs. The algorithm still converges due the fact that in practice we used real number rather than binary number in hidden layer. 

Between different runs, since we start with different intial weights, the final binary representation in hidden layer for a specific number may change. And it is clearly shown that lot of difference exists between runs regarding the binary representation in hidden layer.

# 4. 5-layer NN. The 1st, 2nd, and 3rd hidden layers have 8, 4, and 8 perceptrons, respectively using sigmoid function

In [107]:
compare((8,4,8), 'logistic', False, 0.001)

random intialization  1 :  score is  0.0  ; loss is  3.75071070166
final output
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Accuracy:  0.0

random intialization  2 :  score is  0.0  ; loss is  3.75025456915
final output
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 

# 5. 5-layer NN. The 1st, 2nd, and 3rd hidden layers have 8, 4, and 8 perceptrons, respectively using relu function

In [108]:
compare((8,4,8), 'relu', False, 0.001)

random intialization  1 :  score is  1.0  ; loss is  0.0505047582769
final output
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
Accuracy:  1.0

random intialization  2 :  score is  0.0  ; loss is  3.74048742646
final output
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 

**Discussion**

I cannot get stable states with 5-layer NN using 'sigmoid' function as the activation function. The output and loss indicated that all five runs failed to converge to a local minimum. This is possibly due to the vanishing gradient problem. As we know, each of the neural network's weights receives an update proportional to the gradient of the error function with respect to the current weight in each iteration of training. The problem is that in some cases, the gradient will be vanishingly small, effectively preventing the weight from changing its value. In the worst case, this may completely stop the neural network from further training. The sigmoid function g(z) = 1 / (1 + e^(-z)) have gradient in the range (0,1), and the back propogation computes gradients by the chain rule.This has the effect of multiplying n of these small numbers to compute gradients of the "front" layers in an n-layer network, meaning that the gradient (error signal) decreases exponentially with n while the front layers train very slowly. Since we have multiple layers in the neural networks, it could occur this problem.

However, if I used 'relu' function as the activation function, four out of five runs successfully convered to a local minimum with 100% accuracy. There is no vanishing or exploding gradient problems in relu function (f(x)= max(0,x)). So we could converge to a local minimum. The run 2 failed possibly because the algorithm stuck at some other local minumin with this random initialization